In [ ]:
%load_ext autoreload
%autoreload 2 
%matplotlib inline
# This blokc is important if we want the memory to grow on the GPU, and not block allocate the whole thing
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.utils import plot_model
from keras.callbacks import TensorBoard
from datetime import datetime
import os
from keras.callbacks import EarlyStopping, ModelCheckpoint
from livelossplot.keras import PlotLossesCallback
# Hyperparam opt
import talos as ta

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

# Set path to find modelling tools for later use
import sys
sys.path.append(os.path.join(os.getcwd(),".."))
# Global params live here
import haberrspd.charCNN.globals

In [ ]:
# Load data-loader
from haberrspd.charCNN.auxiliary_tf import create_training_data, LossHistory, binarize
# Load model
from haberrspd.charCNN.models_tf import char_cnn_model, char_cnn_model_talos

## Load training data and validation data as well as auxiliary model parameters

In [ ]:
from pathlib import Path
from sklearn.utils import class_weight 
DATA_ROOT = Path("../data/") / "MJFF" / "preproc" # Note the relative path
# Load training data and auxiliary variables
X_train, X_test, y_train, y_test, max_sentence_length = \
create_training_data(DATA_ROOT,"EnglishData-preprocessed.csv",'sentence')
class_weights = class_weight.compute_class_weight('balanced',list(set(y_train)),y_train)

In [ ]:
# Rev-up tensorboard
# logdir="../logs/char-cnn-keras-" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = TensorBoard(log_dir=logdir)
# Set model
model = char_cnn_model(max_sentence_length)
model.summary()

## Assign a loss function

In [ ]:
loss_func = 'squared_hinge'

if loss_func == 'hinge' or loss_func == 'squared_hinge':
    y_train = [-1 if x==0 else x for x in y_train]
    y_test = [-1 if x==0 else x for x in y_test]
    
if loss_func == 'binary_crossentropy':
    # Check if label-space is correct
    if (-1 in y_train) or (-1 in y_test):
        y_train = [0 if x==-1 else x for x in y_train]
        y_test = [0 if x==-1 else x for x in y_test]

## Train model

In [ ]:
# Compile
model.compile(loss=loss_func,  # TODO: change to cosine loss, cosine_proximity, binary_crossentropy
              optimizer='adam',            # TODO: check which is most appropriate
              metrics=['accuracy'])        # Probs other options here which are more useful

# Check if checkpoints dir exists, if not make it
if not os.path.exists('../../keras_checkpoints'):
    os.makedirs('../../keras_checkpoints')

# Callbacks
file_name = "char-CNN"
check_cb = ModelCheckpoint(file_name + '.{epoch:02d}-{val_loss:.2f}.hdf5',
                           monitor='val_loss',
                           verbose=0,
                           save_best_only=True,
                           mode='min')

earlystop_cb = EarlyStopping(monitor='val_loss',
                             patience=7,
                             verbose=0,
                             mode='auto')

# history = LossHistory()
"""
TODO:

-Add class-weight option to take into account class-imbalance on patients and controls
"""
fit_hist = model.fit(X_train,
                     y_train,
                     validation_data=(X_test, y_test),
                     verbose=0, # Set to zero if using live plotting of losses
                     class_weight = class_weights,
                     batch_size=128,
                     epochs=40,
                     #shuffle=True, # Our data is already shuffled during data loading
                     callbacks=[
                                #check_cb,
                                #tensorboard_callback,
                                PlotLossesCallback(),
                                #earlystop_cb
                               ]
                    )

# TALOS: hyperparameter optimisation

In [ ]:
# Set the parameter space
p = {'conv_output_space' = [16,32,64],
     'number_of_filters' = [1,2,3,4,5,6],
     'filter_length' = [10,15,20],
     'pool_length' = [2,4,8],
     'lr': (0.5, 5, 10),
     'first_neuron':[4, 8, 16, 32, 64],
     'hidden_layers':[0, 1, 2],
     'batch_size': [16,32,64,128],
     'epochs': [16,32,64,128],
     'dropout': (0, 0.5, 5),
     'kernel_initializer': [RandomNormal(mean=0.0, stddev=0.05)],
     'bias_initializer': [RandomNormal(mean=0.0, stddev=0.05)],
     'weight_regulizer':[None],
     'emb_output_dims': [None],
     'shape':['brick','long_funnel'],
     'optimizer': [Adam, Nadam, RMSprop],
     'losses': [logcosh, binary_crossentropy],
     'activation':[relu, elu],
     'last_activation': [sigmoid]}

In [1]:
args = {'a':1,'b':3}

In [12]:
test(4,**args)

8


In [11]:
def test(k,**args):
    print(k+args['a']+args['b'])